# MLB Statcast Data Engineering

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import pybaseball as pb
pb.cache.enable()

## Pull Statcast Data
Trackman was implemented at the start of the 2017 season.

In [2]:
pitch_data = pb.statcast(start_dt = "2017-01-01", end_dt = "2022-12-31")

This is a large query, it may take a moment to complete


  0%|          | 0/1242 [00:00<?, ?it/s]

Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates


100%|██████████| 1242/1242 [11:17<00:00,  1.83it/s]


In [3]:
pitch_data.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
2140,FS,2022-08-23,90.8,-0.70,7.09,"Bautista, Félix",462101,642585,field_out,hit_into_play,...,5,3,5,3,5,Standard,Standard,245,0.011,-0.086
2159,FF,2022-08-23,100.2,-0.56,7.10,"Bautista, Félix",462101,642585,NaN,foul,...,5,3,5,3,5,Standard,Standard,194,0.000,-0.033
2242,FF,2022-08-23,100.5,-0.60,7.09,"Bautista, Félix",462101,642585,NaN,called_strike,...,5,3,5,3,5,Standard,Standard,193,0.000,-0.021
2282,FF,2022-08-23,100.7,-0.69,7.05,"Bautista, Félix",462101,642585,NaN,ball,...,5,3,5,3,5,Standard,Standard,195,0.000,0.023
2353,SL,2022-08-23,87.6,-1.18,6.98,"Bautista, Félix",462101,642585,NaN,ball,...,5,3,5,3,5,Standard,Standard,92,0.000,0.017


## Engineer Additional Fields

### Create `spray_angle` column
In the data we have launch angle and launch speed, but missing spray angle. We can caluclate spray angle with a little trigonometry using the x and y batted ball locations.

To make home plate the point of reference for x and y, constants are subtracted from the raw data points. This allows us to find the spray angle of the batted ball with home plate as the reference.

In [4]:
pitch_data["spray_angle"] = (np.arctan((pitch_data["hc_x"] - 125.42) / (198.27 - pitch_data["hc_y"])) * 180 / np.pi * .75).round(1)

### Create `hit_type` column
The hit type column will be a numerical representation of the batted ball hit type. The number corresponds to the number of total bases associated with a particular hit type. This will be useful when wanting to calculate stats such as SLG% which use total bases.

In [5]:
conditions = [
    (pitch_data["events"] == "single"),
    (pitch_data["events"] == "double"),
    (pitch_data["events"] == "triple"),
    (pitch_data["events"] == "home_run")]
choices = [1, 2, 3, 4]
pitch_data["hit_type"] = np.select(conditions, choices, default = None)

### Create `hit` column
This is a flag column with a binary indicator for whether or not the pitch resulted in a hit.

In [6]:
pitch_data["hit"] = np.where(pitch_data["events"].isin(["single", "double", "triple", "home_run"]), 1, None)

### Create more outcome flags
These are more granular binary indicators for whether or not a pitch resulted in a specific event. These will be useful when wanting to calculate stats such as wOBA.

In [7]:
pitch_data["b1"] = np.where(pitch_data["events"] == "single", 1, None)
pitch_data["b2"] = np.where(pitch_data["events"] == "double", 1, None)
pitch_data["b3"] = np.where(pitch_data["events"] == "triple", 1, None)
pitch_data["hr"] = np.where(pitch_data["events"] == "home_run", 1, None)
pitch_data["ubb"] = np.where(pitch_data["events"] == "walk", 1, None)
pitch_data["ibb"] = np.where(pitch_data["events"] == "intent_walk", 1, None)
pitch_data["hbp"] = np.where(pitch_data["events"] == "hit_by_pitch", 1, None)

In [8]:
pitch_data["so"] = np.where(pitch_data["events"].str.contains("strikeout"), 1, None)
pitch_data["sf"] = np.where(pitch_data["events"].str.contains("sac_fly"), 1, None)
pitch_data["sh"] = np.where(pitch_data["events"].str.contains("sac_bunt"), 1, None)

### Create `ab` column
This is a flag column with a binary indicator for whether or not the pitch resulted in the end of an at-bat. This will be useful, in particular, when wanting to calculate stats that us AB as their denominator.

In [9]:
mask1 = pitch_data["hit"] == 1
mask2 = pitch_data["events"].str.contains("out")
mask3 = pitch_data["events"].str.contains("play")
mask4 = pitch_data["events"].str.contains("error")
pitch_data["ab"] = np.where(mask1 | mask2 | mask3 | mask4, 1, None)

### Create `pa` column
This is a flag column with a binary indicator for whether or not the pitch resulted in the end of a plate appearance. Plate appearances include all at-bats plus errors, walks, sacrifices, hit-by-pitch, and defensive interference. This will be useful, in particular, when wanting to calculate stats that us PA as their denominator.

In [10]:
mask1 = pitch_data["hit"] == 1
mask2 = pitch_data["events"].str.contains("out")
mask3 = pitch_data["events"].str.contains("play")
mask4 = pitch_data["events"].str.contains("error")
mask5 = pitch_data["events"].str.contains("walk")
mask6 = pitch_data["events"].str.contains("sac")
mask7 = pitch_data["events"].isin(["hit_by_pitch", "interf_def"])
pitch_data["pa"] = np.where(mask1 | mask2 | mask3 | mask4 | mask5 | mask6 | mask7, 1, None)

### Create `fielding_team` and `batting_team` columns

In [11]:
pitch_data["fielding_team"] = np.where(pitch_data["inning_topbot"] == "Bot", pitch_data["away_team"], pitch_data["home_team"])
pitch_data["batting_team"] = np.where(pitch_data["inning_topbot"] == "Bot", pitch_data["home_team"], pitch_data["away_team"])